# Team name: ECECH
### Team Leader: Ritik Raj | Email: ritikraj210@gmail.com
### Name: Ansh Trivedi | Email: anshtridev@gmail.com
### Name: Milind Anand | Email: milindanand786@gmail.com


# Spotify Recommendation System


In [ ]:

#importing libraries

import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import sys
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import pairwise_distances





## Extracting data from Spotify web api

In [ ]:

client_id = 'ba80e357e085467f93544999e41c42ff'
client_secret= '2ed50d3ca410418aad8492ad7a82b493'

In [ ]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)
scope = 'playlist-read-private'

In [ ]:
token = util.prompt_for_user_token(scope, client_id= client_id, client_secret=client_secret, redirect_uri='http://localhost:8800/callback/')

In [ ]:
sp = spotipy.Spotify(auth=token)

In [ ]:
#Spotify playlist data used
sp.user_playlist_tracks("Milend", "6sbDkMWxoxObSEsOfOhEiw")

In [ ]:
playlist_creator = "Milend"
playlist_id = "6sbDkMWxoxObSEsOfOhEiw"

In [ ]:
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist", "album", "track_name", "track_id",
                             "danceability", "energy", "key", "loudness", "mode", "speechiness",
                             "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"]
    playlist_df = pd.DataFrame(columns = playlist_features_list)
   
    # Create empty dict
    playlist_features = {}
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Get metadata
        playlist_features["artist_id"] = track["track"]["album"]["artists"][0]["id"]
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        playlist_features["image"] = track["track"]["album"]["images"][0]["url"]
        playlist_features["popularity"] = track["track"]["popularity"]
       
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [ ]:
playlist_df = analyze_playlist(playlist_creator, playlist_id)

In [ ]:

playlist_df.head(17)

In [ ]:
playlist_df.loc[playlist_df['track_name']=='comethru']

In [ ]:
playlist_df.describe()

In [ ]:
from skimage import io
import matplotlib.pyplot as plt

def visualize_songs(df):
    
    
    temp = df['image'].values
    plt.figure(figsize=(15,int(0.625 * len(temp))))
    columns = 5
    
    for i, url in enumerate(temp):
        plt.subplot(len(temp) / columns + 1, columns, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks(color = 'w', fontsize = 0.1)
        plt.yticks(color = 'w', fontsize = 0.1)
        plt.xlabel(df['track_name'].values[i], fontsize = 12)
        plt.ylabel(df['artist'].values[i], fontsize = 12)
        plt.tight_layout(h_pad=0.4, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()

In [ ]:
visualize_songs(playlist_df)

## Recommendation based on popularity

In [ ]:

song_df = playlist_df.sort_values(by=['popularity'], ascending=False)

song_df.head(17)

In [ ]:
#top 10 popular tracks of user's playlist
artist_df = song_df['artist_id'].tolist()[0:10]

In [ ]:
def artist_name(artist_id): 
    artist_features_list = ["artist",  "track_name", "track_id",]
    artist_df = pd.DataFrame(columns = artist_features_list)


    artist_features = {}
    result = sp.artist_top_tracks(artist_id, country='US')['tracks']
    for track in result:

        artist_features["track"] = track["album"]["artists"][0]["name"]
        artist_features["track_id"] = track["id"]
        artist_features["image"] = track["album"]["images"][0]["url"]
        artist_features["popularity"] = track["popularity"]


        track_df = pd.DataFrame(artist_features, index = [0])
        artist_df = pd.concat([artist_df, track_df], ignore_index = True)

    return artist_df['track_id'][0:3]

In [ ]:

print("\n\nTrack ids of the recommended songs")
i=0
for song in artist_df:
    artist_name(song)
    print(artist_name(song))
    i=i+1